<a href="https://colab.research.google.com/github/dogukartal/IBM_AI_Labs/blob/main/Deep%20Learning%20with%20Keras%20and%20Tensorflow/Advanced%20Keras%20Techniques/Hyperparameter_Tuning_with_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [2]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

In [3]:
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Defining the model with hyperparameters


In [4]:
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'), # Integer Search
        Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), # Float Search
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

### Configuring the hyperparameter search


In [5]:
# Create a RandomSearch Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) # Each trail is trained for 5 epochs

# Display a summary of the results
tuner.results_summary()

Trial 10 Complete [00h 00m 46s]
val_accuracy: 0.9593499898910522

Best val_accuracy So Far: 0.9798499941825867
Total elapsed time: 00h 15m 06s
Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 03 summary
Hyperparameters:
units: 224
learning_rate: 0.0009629240377974274
Score: 0.9798499941825867

Trial 01 summary
Hyperparameters:
units: 448
learning_rate: 0.0003307120308325887
Score: 0.9798000156879425

Trial 08 summary
Hyperparameters:
units: 352
learning_rate: 0.001698498750487087
Score: 0.9792500138282776

Trial 02 summary
Hyperparameters:
units: 192
learning_rate: 0.0009249366576101544
Score: 0.978050023317337

Trial 00 summary
Hyperparameters:
units: 224
learning_rate: 0.0025775021806983384
Score: 0.9774999916553497

Trial 05 summary
Hyperparameters:
units: 256
learning_rate: 0.002013780525929294
Score: 0.9765999913215637

Trial 07 summary
Hyperparameters:
units: 32
learning_rate: 0.004706845691073461
Score: 0

In [7]:
# The best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('units'))
print(best_hps.get('learning_rate'))

# Build and Train the Model with Best Hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_val, y_val)
print(f'Test accuracy: {test_acc}')

224
0.0009629240377974274
Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8716 - loss: 0.4401 - val_accuracy: 0.9614 - val_loss: 0.1308
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9672 - loss: 0.1104 - val_accuracy: 0.9697 - val_loss: 0.0984
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9787 - loss: 0.0725 - val_accuracy: 0.9745 - val_loss: 0.0854
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9859 - loss: 0.0484 - val_accuracy: 0.9737 - val_loss: 0.0866
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9887 - loss: 0.0361 - val_accuracy: 0.9725 - val_loss: 0.0952
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9912 - loss: 0.0284 - val_accuracy: 0.9756 - val_loss: 0.0850
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9944 - loss: 0.0196 - val_accuracy: 0.9749 - val_loss: 0.0905
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accur